In [100]:
import lightgbm as lgb
import glob
import numpy as np
import pandas as pd

In [101]:
train = pd.read_csv('data/train.csv')

In [102]:
def calcualte_acc(x, y):
    count = 0
    for i, j in enumerate(x):
        if j == y[i]:
            count += 1
    print(count/len(x))

# beauty

In [103]:
beauty_paths = glob.glob('predictions/beauty/*')

In [104]:
beauty_paths

['predictions/beauty\\conv1d_1',
 'predictions/beauty\\conv1d_2layers',
 'predictions/beauty\\conv1d_3',
 'predictions/beauty\\gru',
 'predictions/beauty\\lstm']

In [105]:
test_paths = glob.glob('predictions/test/*')

In [106]:
test_paths

['predictions/test\\conv1d_1.npy',
 'predictions/test\\conv1d_2layers.npy',
 'predictions/test\\conv1d_3.npy',
 'predictions/test\\gru.npy',
 'predictions/test\\lstm.npy']

In [107]:
test_set = np.load(test_paths[0])[:, :17]
print(test_set.shape)
for i in test_paths[1:]:
    test_set = np.append(test_set, np.load(i)[:, :17], axis=1)

(172402, 17)


In [108]:
test_set = test_set[:76545, :]

In [109]:
num_classes = 17

In [110]:
params={
        "objective" : "multiclass",
        "num_class" : num_classes,
        "num_leaves" : 63,
        "max_depth": -1,
        "learning_rate" : 0.01,
        "bagging_fraction" : 0.9,  # subsample
        "feature_fraction" : 0.9,  # colsample_bytree
        "bagging_freq" : 5,        # subsample_freq
        "bagging_seed" : 2019,
        'max_bin': 63,
        #'device': 'gpu',
        'metric': 'multi_logloss',
        'multi_logloss': 'gbdt'
}

In [111]:
for i in indexes:
    print(i.shape)

(57317,)
(57317,)
(57317,)
(57316,)
(57316,)


In [114]:
for j in range(len(beauty_paths)):
    for i in range(5):
        print(glob.glob(beauty_paths[j]+'/*')[i])
        print(np.load(glob.glob(beauty_paths[j]+'/*')[i]).shape)

predictions/beauty\conv1d_1\beauty_predictions_0.npy
(57317, 17)
predictions/beauty\conv1d_1\beauty_predictions_1.npy
(57317, 17)
predictions/beauty\conv1d_1\beauty_predictions_2.npy
(57317, 17)
predictions/beauty\conv1d_1\beauty_predictions_3.npy
(57316, 17)
predictions/beauty\conv1d_1\beauty_predictions_4.npy
(57316, 17)
predictions/beauty\conv1d_2layers\beauty_predictions_0.npy
(57317, 17)
predictions/beauty\conv1d_2layers\beauty_predictions_1.npy
(57317, 17)
predictions/beauty\conv1d_2layers\beauty_predictions_2.npy
(57317, 17)
predictions/beauty\conv1d_2layers\beauty_predictions_3.npy
(57316, 17)
predictions/beauty\conv1d_2layers\beauty_predictions_4.npy
(57316, 17)
predictions/beauty\conv1d_3\beauty_predictions_0.npy
(57317, 17)
predictions/beauty\conv1d_3\beauty_predictions_1.npy
(57317, 17)
predictions/beauty\conv1d_3\beauty_predictions_2.npy
(57317, 17)
predictions/beauty\conv1d_3\beauty_predictions_3.npy
(57316, 17)
predictions/beauty\conv1d_3\beauty_predictions_4.npy
(57316,

In [68]:
test_output = np.zeros((test_set.shape[0], 17))
for i in range(5):
    print('fold_{}'.format(i))
    lst = [0,1,2,3,4]
    lst.remove(i)

    trn_set = np.load(glob.glob(beauty_paths[0]+'/*')[lst[0]])
    val_set = np.load(glob.glob(beauty_paths[0]+'/*')[i])
    
    for j in beauty_paths[1:]:
        trn_set = np.append(trn_set, np.load(glob.glob(j+'/*')[lst[0]]), axis=1)
        val_set = np.append(val_set, np.load(glob.glob(j+'/*')[i]), axis=1)
    #trn_set = trn_set.reshape(trn_set.shape[0], 1)
        
    for x in lst[1:]:
        one_row = np.load(glob.glob(beauty_paths[0]+'/*')[x])
        for j in beauty_paths[1:]:
            one_row = np.append(one_row, np.load(glob.glob(j+'/*')[x]), axis=1)
        trn_set = np.append(trn_set, one_row, axis=0)
    
    indexes = np.load('np_array/trn_index_b_fold.npy')
    index = int(glob.glob(beauty_paths[0] + '/*')[i][-5])
    print(index)
    val_index = indexes[index]
    other_lst = [0,1,2,3,4]
    other_lst.remove(index)
    trn_index = indexes[other_lst[0]]
    other_lst.remove(other_lst[0])
    for num in other_lst:
        trn_index = np.append(trn_index, indexes[num])
        
    labels_trn = train.iloc[trn_index, 2]
    labels_trn = np.array(labels_trn)

    labels_val = train.iloc[val_index, 2]
    labels_val = np.array(labels_val)
    
    print(trn_set.shape)
    print(val_set.shape)
    lgtrain, lgval = lgb.Dataset(trn_set, labels_trn), lgb.Dataset(val_set, labels_val)
    lgbmodel = lgb.train(params, lgtrain, 2000, valid_sets=[lgtrain, lgval], early_stopping_rounds=100, verbose_eval=100)
    pred = lgbmodel.predict(val_set)
    np.save('first_layer/five_nlp_{}'.format(index), pred)
    pred = lgbmodel.predict(test_set)
    test_output += pred/5

fold_0
0
(229266, 85)
(57317, 85)
Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 1.17311	valid_1's multi_logloss: 1.20785
[200]	training's multi_logloss: 0.810856	valid_1's multi_logloss: 0.869933
[300]	training's multi_logloss: 0.668413	valid_1's multi_logloss: 0.752157
[400]	training's multi_logloss: 0.597806	valid_1's multi_logloss: 0.707006
[500]	training's multi_logloss: 0.554111	valid_1's multi_logloss: 0.687881
[600]	training's multi_logloss: 0.522534	valid_1's multi_logloss: 0.678782
[700]	training's multi_logloss: 0.497822	valid_1's multi_logloss: 0.674467
[800]	training's multi_logloss: 0.477977	valid_1's multi_logloss: 0.672457
[900]	training's multi_logloss: 0.461566	valid_1's multi_logloss: 0.67178
[1000]	training's multi_logloss: 0.447506	valid_1's multi_logloss: 0.671876
Early stopping, best iteration is:
[923]	training's multi_logloss: 0.458147	valid_1's multi_logloss: 0.671741
fold_1
1
(229266, 85)
(57317, 85)
Training un

In [71]:
np.save('first_layer/beauty_five_test.npy', test_output)

In [30]:
calcualte_acc(predictions, labels_val)

0.7924350541724096


In [72]:
b = np.load('first_layer/beauty_five_test.npy')
print(b.shape)
f = np.load('first_layer/fashion_five_test.npy')
print(f.shape)
m = np.load('first_layer/mobile_five_test.npy')
print(m.shape)

(76545, 17)
(55440, 14)
(40417, 27)


In [77]:
output=np.zeros((76545+55440+40417, 58))
output[:76545, :17] = b
output[76545:55440+76545, 17:31] = f
output[55440+76545:, 31:] = m

In [79]:
predictions = []
for i in output:
    predictions.append(list(i).index(i.max()))

In [99]:
np.save('nlp_test.npy', output)

In [85]:
len(predictions)

172402

In [87]:
test = pd.read_csv('data/test.csv')

In [95]:
submission = test[['itemid']]
submission['Category'] = predictions

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [97]:
submission.to_csv('submission.csv')